# Altair plot deconstruction: visualizing the correlation structure of weather data
*Using interactive Altair plots, Python and Pandas to more efficiently explore the correlation structure of a weather dataset.*

## Introduction
Even after singing the praises of plotnine in my [previous article](https://towardsdatascience.com/plotnine-plot-deconstruction-visualizing-the-billboard-hot-100-8048808fd629), it is lacking in one major aspect: interactivity. Luck would have it that an [article on Medium](https://towardsdatascience.com/interactive-election-visualisations-with-altair-85c4c3a306f9) pointed me to a plotting framework that specialises in interacitivity: [Altair](https://altair-viz.github.io/). The style of defining plots has a similar declarative philosophy (visual grammar) as plotnine. You do not specify the individual lines and points, but plot axes and plot layers. This makes the code, in my opinion, more readable and compact. 

The main plot of this article explores the correlation structure of a weather dataset using two connected subplots: a correlation heatmap and a 2d histogram showing the density of points. The following animated gif shows the plot in action:

![](screenshots/animation.gif)

Clicking on a rectangle in the heatmap will show for that particular cell the corresponding data in the 2d histogram. The plot enables you to quickly see what the patterns across correlations are using the heatmap visual, and allows to zoom in on the data underlying the correlation in the 2d histogram.

In this post you will learn about the following topics:

- How to work with Altair and large amounts of data by aggregating the data. 
- How to sort the values on a categorical axis using numerical values in the same data in Altair. This is particlarly useful when the data changes dynamically and you cannot precompute how the values should be sorted. 
- How to dynamically connect the heatmap and the 2d histogram. The main difficulty here is in how to format the data to get this working, see also [this issue](https://github.com/altair-viz/altair/issues/1617) on the Altair Github page.

In the article I will read and process the data and slowly build up both plots. Finally, I will connect the plots together into one cohesive interactive plot. 

## Reading the data
The dataset featured in this article contains climate data I [downloaded](http://projects.knmi.nl/klimatologie/daggegevens/selectie.cgi) from the Royal Dutch Meteorological Institute (KNMI). The following code reads the text file into a Pandas DataFrame:

In [0]:
import pandas as pd
import altair as at
at.data_transformers.disable_max_rows()

def read_knmi_data(filename, names):
    return pd.read_csv(filename, 
                       comment='#',               # Skip all comments
                       header=None,               # The text file contains no header that can be read
                       names=names,               # Set the column names
                       skipinitialspace=True,     # Fix the trailing spaces after the ','-separator
                       parse_dates=[1])           # Let pandas try and transform the second column to a proper datatime object

knmi_data = read_knmi_data('KNMI_20200218.txt', 
                           names=['station', 'datum', 'Wsp_avg', 'Wsp_1hravg', 'Wsp_max', 
                                                      'T_avg', 'T_min', 'T_max', 
                                                      'Sol_duration', 'Global_radiation', 'Precip_total',
                                                      'Precip_hrmax', 'Rel_humid', 'Evaporation'])
knmi_data.head()

,station,datum,Wsp_avg,Wsp_1hravg,Wsp_max,T_avg,T_min,T_max,Sol_duration,Global_radiation,Precip_total,Precip_hrmax,Rel_humid,Evaporation
0,277,2009-02-01,111,130,180,-12,-23,8,51,489,-1,-1,81,5
1,277,2009-02-02,111,140,180,7,-10,20,9,241,0,0,80,3
2,277,2009-02-03,44,80,110,13,1,31,10,294,13,6,91,3
3,277,2009-02-04,20,30,50,9,1,23,5,225,4,3,98,2
4,277,2009-02-05,40,60,90,19,-3,44,0,153,15,10,99,2


The DataFrame contains weather data between 2009 and 2020 for a single weather station: nr 277 (Lauwersoog). Apart from the date (datum) and station id, all the other columns contain measurements of a weather related variable. In this context Wsp stands for windspeed, T for temperature, Precip for precipitation, and humid for humidity. 

## The correlation heatmap
The main goal of this article is to understand the correlation structure of the 12 weather variables in our dataset. What, if any, is the interaction between the various variables in this dataset. We calculate the correlation using the `.corr` function in pandas, which results in a correlation matrix. Note that I cast the data from a [correlation matrix](https://www.displayr.com/what-is-a-correlation-matrix/) to a [long format](https://www.quora.com/What-is-long-and-wide-format-data) dataset using [stack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html), this is needed for plotting in Altair. 

In [0]:
cor_data = (knmi_data.drop(columns=['station'])
              .corr().stack()
              .reset_index()     # The stacking results in an index on the correlation values, we need the index as normal columns for Altair
              .rename(columns={0: 'correlation', 'level_0': 'variable', 'level_1': 'variable2'}))
cor_data['correlation_label'] = cor_data['correlation'].map('{:.2f}'.format)  # Round to 2 decimal
cor_data.head()

,variable,variable2,correlation,correlation_label
0,Wsp_avg,Wsp_avg,1.000000,1.00
1,Wsp_avg,Wsp_1hravg,0.899848,0.90
2,Wsp_avg,Wsp_max,0.844117,0.84
3,Wsp_avg,T_avg,-0.119883,-0.12
4,Wsp_avg,T_min,-0.069923,-0.07


With this data we can make the correlation heatmap:

In [0]:
base = at.Chart(cor_data).encode(
    x='variable2:O',
    y='variable:O'    
)

# Text layer with correlation labels
# Colors are for easier readability
text = base.mark_text().encode(
    text='correlation_label',
    color=at.condition(
        at.datum.correlation > 0.5, 
        at.value('white'),
        at.value('black')
    )
)

# The correlation heatmap itself
cor_plot = base.mark_rect().encode(
    color='correlation:Q'
)

cor_plot + text # The '+' means overlaying the text and rect layer

![](cor_heatmap.png)

The plot consists of two layers: a text layer (text) with the correlations and a rect layer (cor_plot) where the color corresponds to the correlation. The label shows you the actual correlation, the rect layer the pattern in the correlation. 

## Preparing the data for dynamically updating the 2d histogram
Now we have the correlation heatmap done, we can shift our attention to the 2d histogram. To be able to draw the 2d histogram we need a dataset that has the following structure:

- One column with the binned values of one variable for the x-axis
- One column column with the binned values of the other variable for the y-axis
- The density of points for that particular binned combination of variables

To be able to dynamically change the contents of the 2d histogram we need to repeat these three columns for each of the variable combinations in the dataset. As with the correlation plot, we use a long format for the data. To get the combinations of each variable we:

- cast the data from wide to long format
- merge the original data with that long format
- cast to long format for a second time 

The following illlustation shows what happens for a simple example:

![](illustratie_dubbel_merge.png)

Performing these steps for our climate dataset yields the following result:

In [0]:
# Notice the melt-merge-melt combo.
knmi_data_long = knmi_data.melt(id_vars=['station', 'datum'])
knmi_data_long = knmi_data_long.merge(knmi_data, 
                     on=['station', 'datum']).melt(id_vars=['station', 'datum', 'variable', 'value'],
                                                   var_name='variable2', value_name='value2')
knmi_data_long.head()

,station,datum,variable,value,variable2,value2
0,277,2009-02-01,Wsp_avg,111,Wsp_avg,111
1,277,2009-02-01,Wsp_1hravg,130,Wsp_avg,111
2,277,2009-02-01,Wsp_max,180,Wsp_avg,111
3,277,2009-02-01,T_avg,-12,Wsp_avg,111
4,277,2009-02-01,T_min,-23,Wsp_avg,111


To get the data for a partciular combination of variables, we subset on the variable and variable2 colums. For example,

In [0]:
knmi_data_long.query('(variable == "Rel_humid") & (variable2 == "Precip_hrmax")').head()

,station,datum,variable,value,variable2,value2
435790,277,2009-02-01,Rel_humid,81,Precip_hrmax,-1
435802,277,2009-02-02,Rel_humid,80,Precip_hrmax,0
435814,277,2009-02-03,Rel_humid,91,Precip_hrmax,6
435826,277,2009-02-04,Rel_humid,98,Precip_hrmax,3
435838,277,2009-02-05,Rel_humid,99,Precip_hrmax,10


shows the data we need for the Rel_humid versus Precip_hrmax variables. Note that the data has *not* been binned yet.

## Altair and a lot of data: 2d binning
Now we have the raw data ready, we can start binning the data for the 2d histogram. Altair can perform the binning on the fly, but with the amount of data we have the Altair plot becomes very slow. To speed up the plot, we precompute the 2d histogram. We do this using numpy.histogram2d. We first define a function which performs the binning, and casts the data to the long format required for Altair:

In [0]:
import numpy as np

def compute_2d_histogram(var1, var2, df, density=True):
    H, xedges, yedges = np.histogram2d(df[var1], df[var2], density=density)
    H[H == 0] = np.nan

    # Create a nice variable that shows the bin boundaries
    xedges = pd.Series(['{0:.4g}'.format(num) for num in xedges])
    xedges = pd.DataFrame({"a": xedges.shift(), "b": xedges}).dropna().agg(' - '.join, axis=1)
    yedges = pd.Series(['{0:.4g}'.format(num) for num in yedges])
    yedges = pd.DataFrame({"a": yedges.shift(), "b": yedges}).dropna().agg(' - '.join, axis=1)

    # Cast to long format using melt
    res = pd.DataFrame(H, 
                       index=yedges, 
                       columns=xedges).reset_index().melt(
                            id_vars='index'
                       ).rename(columns={'index': 'value2', 
                                         'value': 'count',
                                         'variable': 'value'})
    

    # Also add the raw left boundary of the bin as a column, will be used to sort the axis labels later
    res['raw_left_value'] = res['value'].str.split(' - ').map(lambda x: x[0]).astype(float)   
    res['raw_left_value2'] = res['value2'].str.split(' - ').map(lambda x: x[0]).astype(float) 
    res['variable'] = var1
    res['variable2'] = var2 
    return res.dropna() # Drop all combinations for which no values where found

Then we use this function to calculate the binned 2d data for each of the combinations of variables:

In [0]:
# Use the function for each combination of variables. 
value_columns = knmi_data.columns.drop(['station', 'datum'])
knmi_data_2dbinned = pd.concat([compute_2d_histogram(var1, var2, knmi_data) for var1 in value_columns for var2 in value_columns])
knmi_data_2dbinned.head()

,value2,value,count,raw_left_value,raw_left_value2,variable,variable2
0,15 - 31.2,15 - 31.2,0.000233,15.0,15.0,Wsp_avg,Wsp_avg
11,31.2 - 47.4,31.2 - 47.4,0.000924,31.2,31.2,Wsp_avg,Wsp_avg
22,47.4 - 63.6,47.4 - 63.6,0.001111,47.4,47.4,Wsp_avg,Wsp_avg
33,63.6 - 79.8,63.6 - 79.8,0.000748,63.6,63.6,Wsp_avg,Wsp_avg
44,79.8 - 96,79.8 - 96,0.000433,79.8,79.8,Wsp_avg,Wsp_avg


Also notice the raw_left_value columns, these are the left hand side of the binning interval. These will be used to sort the axis labels correctly in the plot.  

From this data, this yields the following 2d histogram for Rel_humid versus Precip_hrmax:

In [0]:
relhumid_vs_preciphrmax = knmi_data_2dbinned.query('(variable == "Precip_hrmax") & (variable2 == "Rel_humid")')
scat_plot = at.Chart(relhumid_vs_preciphrmax).mark_rect().encode(
    at.X('value:N', sort=at.EncodingSortField(field='raw_left_value')), 
    at.Y('value2:N', sort=at.EncodingSortField(field='raw_left_value2', order='descending')),
    at.Color('count:Q', scale=at.Scale(scheme='blues'))
)

![](2dhist.png)

Which nicely illustrates the low correlation of 0.17 between these two variables. Also notice the use of `EncodingSortField` to correctly order the labels on the x- and y-axis based on the raw_left_value variables. Normally these would be alphabetical, but now they are from small to big. 

## Combining both plots 
Now we have both plots setup, we need to connect both. The connection is done via a selector, in this case selection_single:

In [0]:
var_sel_cor = at.selection_single(fields=['variable', 'variable2'], clear=False, 
                                  init={'variable': 'Evaporation', 'variable2': 'T_max'})

We couple the selector to the variable and variable2 fields in the datasets (corr and binned 2d), and initialize the selector to Evaporation and T_max for variable and variable2 respectively. Now all we have to do is:

- Couple the selector to the correlation heatmap. This means that if I click on the heatmap, it registers which of the value for variable and variable2 are clicked on.
- Use the selector to subset the 2d histogram data before drawing the plot. This automates what we manually did with a call to `knmi_data_2dbinned.quqery()` earlier. 

Subsetting the 2d binned data is done using `transform_filter`:

In [0]:
# NOT RUN
at.Chart(knmi_data_2dbinned).transform_filter(
    var_sel_cor
).mark_rect().etc

The following code defines all the plots, connects them via the selector and yields the desired connected interactive graph:

In [0]:
# Define selector
var_sel_cor = at.selection_single(fields=['variable', 'variable2'], clear=False, 
                                  init={'variable': 'Evaporation', 'variable2': 'T_max'})

# Define correlation heatmap
base = at.Chart(cor_data).encode(
    x='variable2:O',
    y='variable:O'    
)

text = base.mark_text().encode(
    text='correlation_label',
    color=at.condition(
        at.datum.correlation > 0.5, 
        at.value('white'),
        at.value('black')
    )
)

cor_plot = base.mark_rect().encode(
    color=at.condition(var_sel_cor, at.value('pink'), 'correlation:Q')
).add_selection(var_sel_cor)

# Define 2d binned histogram plot
scat_plot = at.Chart(knmi_data_2dbinned).transform_filter(
    var_sel_cor
).mark_rect().encode(
    at.X('value:N', sort=at.EncodingSortField(field='raw_left_value')), 
    at.Y('value2:N', sort=at.EncodingSortField(field='raw_left_value2', order='descending')),
    at.Color('count:Q', scale=at.Scale(scheme='blues'))
)

# Combine all plots. hconcat plots both side-by-side 
at.hconcat((cor_plot + text).properties(width=350, height=350), scat_plot.properties(width=350, height=350)).resolve_scale(color='independent')

alt.HConcatChart(...)

This final graph allows you to freely explore the correlation structure of the data, and yields some interesting insights:

- `Global_radiation` versus `Sol duration` shows a high correlation of 0.87. The 2d histogram supports this, but also nuances the picture. Most of the points are located to the left-bottom edge of the plot. This indicates that the correlation leans heavily on the tail of the data. 
- `Wsp_1hravg` versus `Precip_total` shows a low correlation of 0.31, but the 2d histogram shows that there is a signal: during higher windspeeds an increase in windspeed seems correlated with an increase in precipitation. The correlation is mostly weighed down by the lower windspeeds. 

This allows the reader to easily spot trends in correlation, but also drill down into the details of one specific correlation. 

A flaw of the plot right now is that the axis titles are not yet updated. The best way right now is to just remember that value and variable belong together and value2 and variable2. But makes this more explicit is a nice excercise I leave to the reader.  